In [1]:

bucket_name = 'hawaii-marine-debris'
file_path_template = "{}/jpg/{}"
downloaded_tile = set()
csv_rows = []
islands= ['niihau']#'lanai','bigisland',''
local_tmp_folder = "/tmp/{}"
local_tmp_image_folder = "/tmp/images/{island}/{filename}"
tile_with_debris = []


In [4]:
mkdir /tmp/niihau/

In [5]:
import os
import boto3 
s3_client = boto3.client('s3',
            aws_access_key_id=os.environ.get('hawaii_debris_access_key'),
            aws_secret_access_key=os.environ.get('hawaii_debris_secret_key'))
s3_resource = boto3.resource('s3',aws_access_key_id=os.environ.get('hawaii_debris_access_key'),
            aws_secret_access_key=os.environ.get('hawaii_debris_secret_key'))
    
s3_resource.Bucket(bucket_name).download_file("niihau/deduped_annotations.csv", "/tmp/niihau/deduped_annotations.csv")
def download_image_from_s3(s3_key, island='niihau'):
    basename = os.path.basename(s3_key)
    local_file_path = local_tmp_image_folder.format(island=island, filename=basename)
    s3_resource.Bucket(bucket_name).download_file(s3_key, local_file_path)
    return local_file_path

In [10]:
import pandas as pd
df  =pd.read_csv("/tmp/niihau/deduped_annotations.csv")

df['cleaner_label'] = df['label'].replace({'B':'Buoys','C': 'Cloth','F':'Foam','L':'Line','M':'Metal','N':'Net','P':'Plastic','T':'Tire','W':'wood','V':'Boat'})
# Bicycle,0
# Boat,1
# Surfboard,2
# Bottle,3
# Plastic bag,4
# Wheel,5
# Tire,6
# Buoys,7
# Cloth,8
# Foam,9
# Line,10
# Metal,11
# Net,12
# Plastic,13
# Processedwood,14
# Other,15

# Debris categories:
# B = Buoys and floats
# C = Cloth 
# F = Foam 
# L = Line (single pieces of rope, not net)
# M = Metal
# N = Net
# P = Plastic
# T = Tire
# W = Processed wood
# V = Vessel
# O = Other 

df

,s3_key,x0,y0,x1,y1,label,cleaner_label
0,niihau/jpg/1_1920_04_07.jpg,474,667,524,617,P,Plastic
1,niihau/jpg/1_1920_06_12.jpg,940,393,990,343,B,Buoys
2,niihau/jpg/1_1920_07_13.jpg,242,529,292,479,P,Plastic
3,niihau/jpg/1_1920_07_12.jpg,638,801,738,701,N,Net
4,niihau/jpg/1_1920_07_13.jpg,187,933,237,883,B,Buoys
5,niihau/jpg/1_1920_05_11.jpg,528,991,578,941,B,Buoys
6,niihau/jpg/1_1920_08_13.jpg,284,361,334,311,N,Net
7,niihau/jpg/1_1920_08_13.jpg,263,410,313,360,N,Net
8,niihau/jpg/1_1920_08_13.jpg,511,407,561,357,B,Buoys
9,niihau/jpg/1_1920_08_13.jpg,541,479,591,429,B,Buoys


In [7]:
df.groupby('label').count()

,s3_key,x0,y0,x1,y1
label,,,,,
B,1999,1999,1999,1999,1999
C,19,19,19,19,19
M,25,25,25,25,25
N,742,742,742,742,742
P,3658,3658,3658,3658,3658
T,306,306,306,306,306
V,12,12,12,12,12


In [11]:
def local_file_path(s3_key, island='niihau'):
    basename = os.path.basename(s3_key)
    local_file_path = local_tmp_image_folder.format(island=island, filename=basename)
    return local_file_path
df['local_file_path'] = df['s3_key'].apply(local_file_path)

In [12]:
import numpy as np
msk = np.random.rand(len(df)) < 0.9

train = df[msk]
test = df[~msk]

train[['local_file_path','x0','y1','x1','y0','cleaner_label']].to_csv('keras_annotations_train.csv', encoding='utf-8', header=False, index=False)
test[['local_file_path','x0','y1','x1','y0','cleaner_label']].to_csv('keras_annotations_test.csv', encoding='utf-8', header=False, index=False)


In [13]:
s3_client.upload_file('keras_annotations_train.csv', bucket_name, 'ml/v2/data/keras_annotations_train.csv')
s3_client.upload_file('keras_annotations_test.csv', bucket_name, 'ml/v2/data/keras_annotations_test.csv')

In [15]:
test[['local_file_path','x0','y1','x1','y0','cleaner_label']]

,local_file_path,x0,y1,x1,y0,cleaner_label
6,/tmp/images/niihau/1_1920_08_13.jpg,284,311,334,361,Net
24,/tmp/images/niihau/1_1920_08_14.jpg,598,490,648,540,Buoys
37,/tmp/images/niihau/1_1920_07_15.jpg,571,642,621,692,Plastic
61,/tmp/images/niihau/1_1920_09_17.jpg,290,928,340,978,Plastic
72,/tmp/images/niihau/1_1920_08_18.jpg,294,361,344,411,Buoys
76,/tmp/images/niihau/1_1920_06_24.jpg,833,265,883,315,Buoys
79,/tmp/images/niihau/1_1920_06_25.jpg,692,585,742,635,Buoys
115,/tmp/images/niihau/2_1747_30_19.jpg,169,454,219,504,Buoys
119,/tmp/images/niihau/2_1747_30_19.jpg,609,308,659,358,Buoys
123,/tmp/images/niihau/2_1747_30_19.jpg,701,402,751,452,Buoys


In [ ]:

# for index, row in df[:20].iterrows():
#     local_file = download_image_from_s3(row['s3_key'])
#     print(local_file)